# Ethanol example

This notebook should provide a useful explanation of the RESP2 python module 
and especially how it can be used. 


We import the resp 2 library and set up the logger.


In [1]:
import resp2
import logging as log
log.getLogger().setLevel(log.INFO)

### This step is optional and only required if a ForceBalance calculations should be performed afterwards.

As a first step we are creating the target directory ethanol-liquid and the corresponding data.csv file.
All values in the data.csv are used in the ForceBalance calculation. This module should also create the corresponding 
pdb files of the monomer and the liquid box. 

In this testcase, density, hov and the dielectric constant is used in ForceBalance

Please not that densities are required to be in g/l, Heats of Vaporization in kcal/mol. 

Footnote: We use a small program which is part of the openforcefield toolkit examples to create the liquid boxes. It sometimes can occur that the atoms are not 
numbered in the same order as the appear, which can cause some troubles during the FB simulation setup.
If you encounter troubles during the simulation setup it is worth to have a look at the corresponding files and have a look 
at the atom order. If this problem occurs you can save a mol2 file with the correct atom numbering
with an S at the end of the filename, e.g. ETHS.mol2 and rerun this step again to overwrite the atom numbering in all files.


In [2]:
resp2.create_target(name ='ethanol',density=789.3,hov = 42.3, dielectric=32.7,smiles='CCO',resname ='ETH')

INFO:root:Created target file ethanol-liquid/data.csv
    Density: 789.3 g/l
    Heat of Vaporization: 42.3 kcal/mol
    Dielectric Constant: 32.7
INFO:root:The following SMILES string will be converted: CCO
INFO:root:-=# Output #=- Created ETH.mol2 containing single molecule
INFO:root:-=# Output #=- Created ETH.pdb containing single molecule
INFO:root:Running gmx insert-molecules to create a solvent box...
INFO:root:Time elapsed:  0.005 seconds
INFO:root:Running gmx insert-molecules to create a solvent box...
INFO:root:Time elapsed:  0.290 seconds
INFO:root:-=# Output #=- Created ETH-box.pdb containing solvent box with 700 molecules and length 4.631


0

The previous step was only to prepare the input for ForceBalance if you do not want to use ForceBalance you can ignore the previous step. 

Now the RESP2 parameterization starts. 

For consistency we are using the monomer file created in the previous step for the parameterization. If you 
avoided the previous step you can easily create a mol2 file by smiles string using the smiles2mol2 function of the RESP2
package. 

We will call create_conformers which uses omega to create a bunch of conformers. For the exact options please look at the corresponding function to
see all values used. The options were chosen as a compromise between speed and accruacy. 

In [3]:
number_of_conformers=resp2.create_conformers(infile='ETH.mol2', outfile='ETH-conformers.mol2', folder = 'ethanol-liquid')

INFO:root:Created conformations for ethanol-liquid/ETH.mol2 and saved them to ethanol-liquid/ETH-conformers.mol2


The previous function should have created a bunch of ETH-conformer files. These are non-optimized structures, which we want 
to optimize in the next step.

The optimize_conformers function is optimizing and copying the resulting files to xyz format. If opt = False 
the function is only copying the file with is current geometry to an xyz format.
The files of the xyz files are chosen to be automatically found by the next functions.

In [4]:
resp2.optimize_conformers(name ='ethanol',resname ='ETH', opt = True,number_of_conformers=number_of_conformers)

After we have all optimized geometries we can actually start to create charges. Therefore we use the help
ot the respyte program package which was developed by Lee-Ping Wang and co-workers fro UC Davies. 
To do so we are using the create_respyte function which creates all input files and run all necessary calculations.
We are running it with 3 different type options: RESP1, RESP2GAS, RESP2LIQUID.

To use scaled RESP1 charges we do only require the calculation of type RESP1.
For RESP2 parameterization RESP2GAS and RESP2LIQUID are both required.

In [5]:
resp2.create_respyte(name ='ethanol',resname='ETH', type='RESP2LIQUID',number_of_conformers=number_of_conformers)
resp2.create_respyte(name ='ethanol', resname='ETH', type='RESP2GAS',number_of_conformers=number_of_conformers)
resp2.create_respyte(name ='ethanol', resname = 'ETH',type='RESP1',number_of_conformers=number_of_conformers)

INFO:root:Create folder structure for ethanol with 0 conformers
INFO:root:Create folder structure for ethanol with 0 conformers
INFO:root:Create folder structure for ethanol with 0 conformers


0

The previous comment shoud have created 3 new folders with the output of the respyte calculations. Please to not rename these folders as
the next function is looking for exactly these folder and the respyte ouptut in the folder.

The next function is combining the RESP calculations of the previous step to calculate charges. The first line is
scaling RESP1 charges. IN this case all charges are scaled by a factor 1.05. So all charges
are 5% higher than for regular RESP1

The second line is creating a mixture of gas phase and liquid phase charges. We are using here a mixing paramter of 
0.5 which corresponds to 50% gas charges and 50% liquid charges. With this mixing we can adjust the polarity of our 
forcefield and parameters, respectively. 

In [6]:
resp2.create_charge_file(name='ethanol',resname ='ETH', type = 'RESP1', delta= 1.05)
resp2.create_charge_file(name='ethanol',resname ='ETH', type = 'RESP2', delta= 0.5)

INFO:root:Created charges RESP1 type charges with a delta value of 1.05
INFO:root:Created charges RESP2 type charges with a delta value of 0.5


0

Although, it is useful to have the different functions available through an python API, a user may is just
interested in creating RESP2 charges with a single command therefore, we wrapped all the command shown above in a single function 
to allow RESP2 charges to be generated very easily.
The following commend is doing exactly what is done in the notebook. 

In [7]:
resp2.create_target(name='methanol', density=789.3, hov=42.3, dielectric=32.7, smiles='CO', resname='MET')
resp2.create_RESP2(opt = True, name = 'methanol', resname = 'MET', folder = 'methanol-liquid'  )
#create_RESP2(

INFO:root:Created target file methanol-liquid/data.csv
    Density: 789.3 g/l
    Heat of Vaporization: 42.3 kcal/mol
    Dielectric Constant: 32.7
INFO:root:The following SMILES string will be converted: CO
INFO:root:-=# Output #=- Created MET.mol2 containing single molecule
INFO:root:-=# Output #=- Created MET.pdb containing single molecule
INFO:root:Running gmx insert-molecules to create a solvent box...
INFO:root:Time elapsed:  0.004 seconds
INFO:root:Running gmx insert-molecules to create a solvent box...
INFO:root:Time elapsed:  0.112 seconds
INFO:root:-=# Output #=- Created MET-box.pdb containing solvent box with 700 molecules and length 4.103
INFO:root:Created conformations for methanol-liquid/methanol-liquid/MET.mol2 and saved them to methanol-liquid/methanol-liquid/MET-conformers.mol2
INFO:root:Create folder structure for methanol with 0 conformers
INFO:root:Create folder structure for methanol with 0 conformers
INFO:root:Create folder structure for methanol with 0 conformers

methanol


0

In [ ]:
import os
#os.mkdir('butanol-liquid')
#resp2.create_smifile_from_string(smiles='CCO',filename='butanol-liquid/BUT.mol2')
resp2.create_RESP2(opt = True, name = 'butanol', resname = 'BUT', folder = 'butanol-liquid' )



